In [1]:
# base imports
import numpy as np
import pandas as pd
import xgboost

# import helpers
import sys
sys.path.append("../../..")
from utils.data import *
from utils.helpers import *
from utils.setup import Config

In [2]:
book_df, trade_df = get_trade_and_book_by_stock_and_time_id(0, 5)
book_df.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
0,5,0,1.001422,1.002301,1.00137,1.002353,3,226,2,100,0
1,5,1,1.001422,1.002301,1.00137,1.002353,3,100,2,100,0
2,5,5,1.001422,1.002301,1.00137,1.002405,3,100,2,100,0
3,5,6,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0
4,5,7,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0


In [9]:
"""
Logic behind the below function is that if at the end of the period we see a spike in volatility then this is more likely to continue in the next period
than if the volatility spike was observed in the first 20% for example.
"""

def calc_vol_increase(book_df: pd.DataFrame, rolling_window: int = 20):
    book_df["vwap"] = best_vwap(book_df)
    vol_df = book_df.rolling(rolling_window).apply(realized_volatility)

    #  calculate the difference between midway in the time-period to the end period
    reference_vol = vol_df.iloc[int(len(vol_df)/2)]["vwap"]
    end_vol = vol_df.iloc[-1]["vwap"]

    return (end_vol / reference_vol) - 1

In [10]:
calc_vol_increase(book_df)  # vol is 25% at the end than the midpoint

0.25169939498484895